In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
dataset = pd.read_csv('data/data_clean_removed.csv')
dataset.drop(["Unnamed: 0"],inplace= True,axis = 1)

In [3]:
dataset = dataset.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


In [5]:
dataset.replace({'elevatoe': {1: 'jest', 0: 'nie ma'}}, inplace=True)
dataset.replace({'Parkingplace': {1: 'jest', 0: 'nie ma'}}, inplace=True)
dataset.replace({'balkon': {1: 'jest', 0: 'nie ma'}}, inplace=True)
dataset.replace({'taras': {1: 'jest', 0: 'nie ma'}}, inplace=True)
dataset.replace({'ogrdek': {1: 'jest', 0: 'nie ma'}}, inplace=True)

In [6]:
dataset.head(4)

,Price,Space,Rooms,Market,Year,elevator,Parkingplace,balkon,taras,ogrdek,district,street,level,max_level
0,4000000,560,4,wtórny,2015,1,jest,jest,jest,jest,Wawer,Patriotów,3,3
1,4000000,560,4,wtórny,2015,1,jest,jest,jest,jest,Wawer,Patriotów,3,3
2,21350100,487,10,wtórny,2021,1,jest,nie ma,nie ma,nie ma,Śródmieście,Elektryczna,6,6
3,19900000,487,6,wtórny,2001,1,jest,nie ma,nie ma,nie ma,Mokotów,Cybulskiego,5,5


In [13]:
dataset_dummies = pd.get_dummies(dataset,columns=['Market','street','district','Parkingplace','balkon','taras','ogrdek','elevator'])

In [14]:
dataset_dummies.head(10)

,Price,Space,Rooms,Year,level,max_level,Market_pierwotny,Market_wtórny,street_ Jana Pawła Woronicza,street_ Skoroszewska,...,Parkingplace_jest,Parkingplace_nie ma,balkon_jest,balkon_nie ma,taras_jest,taras_nie ma,ogrdek_jest,ogrdek_nie ma,elevator_0,elevator_1
0,4000000,560,4,2015,3,3,0,1,0,0,...,1,0,1,0,1,0,1,0,0,1
1,4000000,560,4,2015,3,3,0,1,0,0,...,1,0,1,0,1,0,1,0,0,1
2,21350100,487,10,2021,6,6,0,1,0,0,...,1,0,0,1,0,1,0,1,0,1
3,19900000,487,6,2001,5,5,0,1,0,0,...,1,0,0,1,0,1,0,1,0,1
4,795900,445,2,2022,4,8,1,0,0,0,...,1,0,0,1,0,1,0,1,0,1
5,4500000,423,10,1936,2,2,0,1,0,0,...,1,0,0,1,0,1,0,1,1,0
6,6200000,350,6,2009,3,3,0,1,0,0,...,1,0,0,1,0,1,0,1,0,1
7,15671000,344,6,2009,1,7,0,1,0,0,...,1,0,0,1,0,1,0,1,0,1
8,6500000,328,5,2022,11,16,0,1,0,0,...,1,0,0,1,0,1,0,1,0,1
9,4850000,320,5,1912,6,6,0,1,0,0,...,0,1,0,1,0,1,0,1,0,1


In [15]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error


x_train,x_test,y_train,y_test = train_test_split(dataset_dummies.drop(['Price'],axis=1),dataset_dummies['Price'],test_size=0.2,random_state=42)

In [16]:
gbr  = GradientBoostingRegressor()
params = {'n_estimators':[100,200,300,400,500,600,700,800,900,1000],'max_depth':[1,2,3,4,5,6,7,8,9,10],'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
search = RandomizedSearchCV(gbr,param_distributions=params,cv=5,return_train_score=False)

search.fit(x_train,y_train)

RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(),
                   param_distributions={'learning_rate': [0.1, 0.2, 0.3, 0.4,
                                                          0.5, 0.6, 0.7, 0.8,
                                                          0.9, 1],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]})

In [17]:
best_gbr = search.best_estimator_

In [20]:
from sklearn.model_selection import cross_val_score
cross_val_score(best_gbr,x_test, y_test,scoring = 'neg_root_mean_squared_error',cv = 10).mean()

-384251.2458873785

In [21]:
import pickle

filename = 'gbr_model.sav'
pickle.dump(best_gbr, open(filename, 'wb'))